In [4]:
import pandas as pd
import os
import glob
import gc

In [ ]:
df = pd.read_csv('../../data/economic.csv')
df.head(10).T

,0,1,2
YEAR,2011.000000,2011.000000,2011.000000
STATE_FIPS,37.000000,37.000000,37.000000
COUNTY_FIPS,43.000000,51.000000,81.000000
MEDIAN_HOUSEHOLD_INCOME,36711.000000,44861.000000,46288.000000
POP_POVERTY_DETERMINED,10380.000000,302057.000000,469463.000000
POP_BELOW_POVERTY,2262.000000,50175.000000,76141.000000
POP_16_PLUS,8680.000000,240096.000000,382682.000000
POP_UNEMPLOYED,4258.000000,78349.000000,125748.000000
HOUSEHOLDS_TOTAL,4464.000000,118117.000000,192064.000000
HOUSEHOLDS_SNAP,571.000000,15819.000000,20913.000000


In [2]:
def combine_cbp_data(directory='.', output_file='cbp_data.csv'):
    """
    Reads all cbpYYco.txt files in the specified directory,
    adds a 'YEAR' column, and combines them into a single DataFrame.
    """
    # Pattern to match files like cbp07co.txt, cbp14co.txt in the directory
    file_pattern = os.path.join(directory, 'cbp??co.txt')
    files = glob.glob(file_pattern)

    if not files:
        print(f"No files found matching pattern 'cbp??co.txt' in '{directory}'")
        return None

    # Define dtype mapping to preserve leading zeros in FIPS codes
    dtype_map = {
        'FIPSTATE': str,
        'FIPSCTY': str,
        'CENSTATE': str,
        'CENCTY': str,
        'NAICS': str,
        'FIPSTATE'.lower(): str,
        'FIPSCTY'.lower(): str,
        'CENSTATE'.lower(): str,
        'CENCTY'.lower(): str,
        'NAICS'.lower(): str
    }

    column_mapping = {
        "FIPSTATE": "State_FIPS",
        "FIPSCTY": "County_FIPS",
        "NAICS": "NAICS_Industry_Code",
        "EMPFLAG": "Emp_Suppression_Flag",
        "EMP_NF": "Emp_Noise_Flag",
        "EMP": "Total_Employees",
        "QP1_NF": "Q1_Payroll_Noise_Flag",
        "QP1": "Q1_Payroll_1000s",
        "AP_NF": "Annual_Payroll_Noise_Flag",
        "AP": "Annual_Payroll_1000s",
        "EST": "Total_Establishments",
        "N1_4": "Est_1_to_4_Emp",
        "N<5": "Est_1_to_4_Emp",       # Handling the 2017+ schema change
        "N5_9": "Est_5_to_9_Emp",
        "N10_19": "Est_10_to_19_Emp",
        "N20_49": "Est_20_to_49_Emp",
        "N50_99": "Est_50_to_99_Emp",
        "N100_249": "Est_100_to_249_Emp",
        "N250_499": "Est_250_to_499_Emp",
        "N500_999": "Est_500_to_999_Emp",
        "N1000": "Est_1000_Plus_Emp",
        "N1000_1": "Est_1000_to_1499_Emp",
        "N1000_2": "Est_1500_to_2499_Emp",
        "N1000_3": "Est_2500_to_4999_Emp",
        "N1000_4": "Est_5000_Plus_Emp",
        "CENSTATE": "Census_State_Code",
        "CENCTY": "Census_County_Code"
    }

    for filepath in sorted(files):
        filename = os.path.basename(filepath)
        try:
            # Extract 2-digit year from 'cbpYYco.txt' (indices 3 and 4)
            yy = filename[3:5]
            year = 2000 + int(yy)
            
            print(f"Processing {filename} (Year: {year})...")

            # Read the CSV
            # Remove dtype_map from read_csv to avoid mismatch issues if headers are lowercase
            df = pd.read_csv(filepath, encoding='latin1', low_memory=True, dtype=dtype_map)
            df.columns = df.columns.str.upper()
            df.rename(columns=column_mapping, inplace=True)

            # Add context columns
            df['YEAR'] = year
            
            # Create full FIPS code (State + County)
            # Force conversion to string (.astype(str)) before using .str accessor
            if 'FIPSTATE' in df.columns and 'FIPSCTY' in df.columns:
                df['FIPS'] = df['FIPSTATE'].astype(str).str.zfill(2) + df['FIPSCTY'].astype(str).str.zfill(3)

            file_exists = os.path.exists(output_file)
            df.to_csv(output_file, mode='a', index=False, header=not file_exists)
            del df
            gc.collect()

        except Exception as e:
            print(f"Error reading {filename}: {e}")

combine_cbp_data('../../data/cbp')

Processing cbp09co.txt (Year: 2009)...
Processing cbp10co.txt (Year: 2010)...
Processing cbp11co.txt (Year: 2011)...
Processing cbp12co.txt (Year: 2012)...
Processing cbp13co.txt (Year: 2013)...
Processing cbp14co.txt (Year: 2014)...
Processing cbp15co.txt (Year: 2015)...
Processing cbp16co.txt (Year: 2016)...
Processing cbp17co.txt (Year: 2017)...
Processing cbp18co.txt (Year: 2018)...
Processing cbp19co.txt (Year: 2019)...


AttributeError: 'NoneType' object has no attribute 'rename'

In [ ]:
col_specs = [
    (0, 4),    # Year
    (4, 6),    # State Postal Abbreviation
    (6, 8),    # State FIPS
    (8, 11),   # County FIPS
    (13, 14),  # Race (Notice skip of indices 11-13)
    (14, 15),  # Origin
    (15, 16),  # Sex
    (16, 18),  # Age
    (18, 26)   # Population
]

names = [
    'Year', 
    'State_Abbr', 
    'State_FIPS', 
    'County_FIPS', 
    'Race', 
    'Origin', 
    'Sex', 
    'Age', 
    'Population'
]

# Define data types to ensure FIPS codes retain leading zeros
# and numeric fields are parsed correctly
dtype_mapping = {
    'Year': int,
    'State_Abbr': str,
    'State_FIPS': str, 
    'County_FIPS': str,
    'Race': int,
    'Origin': int,
    'Sex': int,
    'Age': int,
    'Population': int
}

print("Starting...")
chunk_size = 100000
chunks = []
reader = pd.read_fwf(
    '../../data/pop.txt',
    colspecs=col_specs,
    names=names,
    dtype=dtype_mapping,
    header=None,
    chunksize = chunk_size
    )

for i, chunk in enumerate(reader):
    print(f"Processing samples: {i*chunk_size} to {(i+1)*chunk_size}", end='\r')
    filtered_chunk = chunk[chunk['Year'] >= 2009]
    chunks.append(filtered_chunk)
df = pd.concat(chunks, ignore_index=True)

if df is not None:
    df.to_csv('../../data/pop.csv', index=False)
    df.head(3).T

Starting...
Processing samples: 0 to 100000
Processing samples: 100000 to 200000
Processing samples: 200000 to 300000
Processing samples: 300000 to 400000
Processing samples: 400000 to 500000
Processing samples: 500000 to 600000
Processing samples: 600000 to 700000
Processing samples: 700000 to 800000
Processing samples: 800000 to 900000
Processing samples: 900000 to 1000000
Processing samples: 1000000 to 1100000
Processing samples: 1100000 to 1200000
Processing samples: 1200000 to 1300000
Processing samples: 1300000 to 1400000
Processing samples: 1400000 to 1500000
Processing samples: 1500000 to 1600000
Processing samples: 1600000 to 1700000
Processing samples: 1700000 to 1800000
Processing samples: 1800000 to 1900000
Processing samples: 1900000 to 2000000
Processing samples: 2000000 to 2100000
Processing samples: 2100000 to 2200000
Processing samples: 2200000 to 2300000
Processing samples: 2300000 to 2400000
Processing samples: 2400000 to 2500000
Processing samples: 2500000 to 260000

In [6]:
filepath = '../../data/MMG_county_df_clean.csv'
df = pd.read_csv(filepath)
df.head(3).T

,0,1,2
Year,2018,2018,2018
FIPS,1001.0,1003.0,1005.0
County,autauga,baldwin,barbour
State,AL,AL,AL
Food_Insecurity_Rate,0.156,0.129,0.219
Num_Food_Insecure_Persons,8620.0,26860.0,5650.0
Low_Threshold_State,1.3,1.3,1.3
Low_Threshold_Type,SNAP,SNAP,SNAP
High_Threshold_State,1.85,1.85,1.85
High_Threshold_Type,Other Nutrition Program,Other Nutrition Program,Other Nutrition Program


In [ ]:
def master_merge():
    print("1. Running CBP Combiner (this may take a moment)...")
    try:
        cbp_df = pd.read_csv('cbp_data.csv', low_memory=True)
        
        print("Loading context datasets (Population, Econ, MMG)...")
        pop = pd.read_csv('../../data/pop.csv')
        mmg = pd.read_csv('../../data/MMG_county_df_clean.csv')
    except FileNotFoundError as e:
        print(f"Error loading CSVs: {e}")
        return

    # --- STEP 3: Standardize Linking Keys (FIPS & Year) ---
    print("3. Standardizing FIPS codes and Years...")

    # A. Standardize POP Keys
    # Ensure State is 2 digits, County is 3 digits
    pop['FIPS'] = pop['State_FIPS'].astype(str).str.zfill(2) + \
                  pop['County_FIPS'].astype(str).str.zfill(3)
    pop.rename(columns={'Year': 'YEAR'}, inplace=True)

    # C. Standardize MMG Keys
    # Remove decimals if they exist (1001.0 -> 1001) then pad
    mmg['FIPS'] = mmg['FIPS'].astype(float).astype(int).astype(str).str.zfill(5)
    mmg.rename(columns={'Year': 'YEAR'}, inplace=True)

    # --- STEP 4: Create Unified Context Table ---
    print("4. Creating County Context Table (Merging Pop, MMG, Econ)...")
    
    # Merge Population and Food Insecurity on YEAR and FIPS
    # 'outer' ensures we keep counties even if one dataset is missing a year
    county_context = pd.merge(pop, mmg, on=['YEAR', 'FIPS'], how='outer', indicator=True)
    print("Indicator Column:", county_context['_merge'].value_counts())

    print(f"   Context Table Shape: {county_context.shape}")

    # --- STEP 5: The Master Merge ---
    print("5. Performing Master Merge (CBP + Context)...")
    
    # We merge CBP (Left) with Context (Right). 
    # Using 'left' ensures we keep every Industry record from CBP.
    final_df = pd.merge(
        cbp_df, 
        county_context, 
        on=['YEAR', 'FIPS'], 
        how='left',
        indicator=True
    )
    
    print("Indicator Column:", final_df['_merge'].value_counts())
    print(f"   Final Shape: {final_df.shape}")

    # --- STEP 6: Save Result ---
    output_filename = 'final_master_dataset.csv'
    print(f"6. Saving final dataset to {output_filename}...")
    
    final_df.to_csv(output_filename, index=False)
    
    print("="*40)
    print("SUCCESS!")
    print(f"Saved to: {output_filename}")
    print(f"Final Shape: {final_df.shape}")
    print("="*40)
    print(final_df.head())


master_merge()

1. Running CBP Combiner (this may take a moment)...
